# Tarea 5 - Qlearning
## Simulación de juego de navegación por laberinto, recoger espada, no caer en veneno y alcanzar enemigo final.
## Si bien el problema se dirigió hacia un juego, esta solución podría utilizarse para carritos "inteligentes" para acomodar artículos en bodegas grandes y complejas, de la manera más eficiente.

In [19]:
import numpy as np
class Env:
    def __init__(self):
        self.height = 11
        self.width = 11
        self.posX = 10
        self.posY = 9
        self.has_sword = 0
        self.endX = 5
        self.endY = 0
        self.actions = [0, 1, 2, 3]

    def reset(self):
        self.posX = 10
        self.posY = 9
        self.done = False
        self.has_sword = 0
        return 0, False

    ##### Calcular cuál debe ser la próxima acción: explorar vs explotar #####
    def get_next_action(self, epsilon, qtable, is_training):
        if np.random.random() > epsilon or is_training==False:
            return np.argmax(qtable[self.posX, self.posY, self.has_sword])
        else:
            return np.random.choice(self.actions)

    ##### Realizar un paso con la acción determinada. Calcular el premio y verificar si llegó a un punto final del epoch #####
    def take_step(self, action, rewards):
        if action == 0: # left
            self.posX = self.posX-1 if self.posX > 0 else self.posX
        if action == 1: # right
            self.posX = self.posX+1 if self.posX < self.width - 1 else self.posX
        if action == 2: # up
            self.posY = self.posY-1 if self.posY > 0 else self.posY
        if action == 3: # down
            self.posY = self.posY+1 if self.posY < self.height - 1 else self.posY

        done = (self.posX == self.endX and self.posY == self.endY) or (rewards[self.posX, self.posY]==-100)
        reward = rewards[self.posX,self.posY]
        return reward, done

    ##### Desplegar laberinto y su población #####
    def render(self, rewards, route=[]):
        for i in range(self.height):
            for j in range(self.width):
                if self.posY == i and self.posX == j:
                    print("O", end='')
                elif self.endY == i and self.endX == j:
                    print("T", end='')
                elif rewards[j, i]==-200:
                    print("Ω", end='')
                elif rewards[j, i]==+200:
                    print("┼", end='')
                elif rewards[j, i]==-100:
                    print("█", end='')
                else:
                    print("░", end='')
            print("")

![alt text](images/img.jpg "Title")
## Una vez que se recoge la espada, ya desaparece el premio para ese epoch, y así nos evitamos que se siga visitando esa coordenada indefinidamente.
## En el case del veneno, este sí se mantiene, pero el "agente" aprenderá a evitarlo.
## Todo lo negro es abismo y el "agente" muere.

# Definición de acciones, premios y Qtable

In [20]:
actions = ['left', 'right', 'up', 'down']
env = Env()
qtable = np.random.random((env.width, env.height, 2, len(env.actions)))
rewards = np.full((env.width, env.height), -100)
rewards[0] =  [-100,-100,-100,-100,-100,100 ,-100,-100,-100,-100,-100]
rewards[1] =  [-100,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-100]
rewards[2] =  [-100,-1  ,-100,-100,-100,-100,-100,-1  ,-100,-1  ,-100]
rewards[3] =  [-100,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-100,-1  ,-100]
rewards[4] =  [-100,-100,-100,-1  ,-100,-100,-100,-1  ,-100,-100,-100]
rewards[5] =  [+200,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-200]
rewards[6] =  [-100,-100,-100,-100,-100,-1  ,-100,-100,-100,-100,-100]
rewards[7] =  [-100,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-100]
rewards[8] =  [-100,-100,-100,-1  ,-100,-100,-100,-1  ,-100,-100,-100]
rewards[9] =  [-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ,-1  ]
rewards[10] = [-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100]
rewards=rewards.T

# Definición de híper-parámetros: epochs, factor de descuento, mezcla de exploración vs explotación, epsilon decay, y tasa de aprendizaje

In [21]:
epochs = 3000
epsilon = 0.1 # Qué tan seguido explorar en vez de explotar
gamma = 0.8 # Factor de descuento para temporal difference en Bellman
decay = 0.0001 # epsilon decay % por epoch
learning_rate = 0.9 # ritmo de aprendizaje

# Función de entrenamiento

In [22]:
def recorrer_laberinto(temp_epsilon, is_training):
    best_route=[]
    for i in range(epochs):
        rewards[0,5]=200 # Nos aseguramos que el premio de la espada exista para cada inicio de epoch
        reward, done = env.reset() # "resetiamos" premio actual, posiciones y si está "done"
        steps = 0
        while not done:
            #env.render(rewards) 
            steps += 1 
            action = env.get_next_action(temp_epsilon, qtable, is_training) # obtenemos la próxima acción
            oldX, oldY = env.posX, env.posY # guardamos las coordenadas
            reward, done = env.take_step(action, rewards) # tomamos el paso y obtenemos el reward y si está "done"
            old_qvalue = qtable[oldX, oldY, env.has_sword, action] # guardamos el qvalue anterior

            ##### Equación de Bellman #####
            temporal_difference = reward + (gamma * np.max(qtable[env.posX, env.posY,env.has_sword])) - old_qvalue
            qtable[oldX, oldY, env.has_sword, action] = old_qvalue + (learning_rate * temporal_difference) 
            if reward==200:
                env.has_sword=1
                rewards[env.posX, env.posY]=-1 # el premio por la espada desaparece
            print("epoch #", i+1, "/", epochs, "action:",actions[action])
            best_route.append((oldX, oldY, actions[action]))
        # Entre más aprendemos, menos random
        if env.posX==env.endX and env.posY==env.endY:
            verb = "Won"
        else:
            verb = "Died"
        print("\n"+verb+" in", steps, "steps".format(steps), 'last action:',actions[action])
        temp_epsilon -= decay*temp_epsilon
        if is_training==False: break
    return best_route

# Entrenar 
## Nota. Quitar # de #env.render(rewards) en código de arriba para ver como camina por laberinto.

In [23]:
_ = recorrer_laberinto(epsilon, is_training=True)

epoch # 1 / 3000 action: up

Died in 1 steps last action: up
epoch # 2 / 3000 action: right
epoch # 2 / 3000 action: left
epoch # 2 / 3000 action: right
epoch # 2 / 3000 action: down

Died in 4 steps last action: down
epoch # 3 / 3000 action: left
epoch # 3 / 3000 action: down

Died in 2 steps last action: down
epoch # 4 / 3000 action: right
epoch # 4 / 3000 action: left
epoch # 4 / 3000 action: up

Died in 3 steps last action: up
epoch # 5 / 3000 action: left
epoch # 5 / 3000 action: down

Died in 2 steps last action: down
epoch # 6 / 3000 action: left
epoch # 6 / 3000 action: left
epoch # 6 / 3000 action: up

Died in 3 steps last action: up
epoch # 7 / 3000 action: left
epoch # 7 / 3000 action: left
epoch # 7 / 3000 action: left
epoch # 7 / 3000 action: up
epoch # 7 / 3000 action: up
epoch # 7 / 3000 action: down
epoch # 7 / 3000 action: down
epoch # 7 / 3000 action: down

Died in 8 steps last action: down
epoch # 8 / 3000 action: left
epoch # 8 / 3000 action: left
epoch # 8 / 3000 a

# Encontrar mejor ruta

In [24]:
route = recorrer_laberinto(0, is_training=False)
route

epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: up
epoch # 1 / 3000 action: up
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: up
epoch # 1 / 3000 action: up
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: right
epoch # 1 / 3000 action: right
epoch # 1 / 3000 action: right
epoch # 1 / 3000 action: right
epoch # 1 / 3000 action: right
epoch # 1 / 3000 action: right
epoch # 1 / 3000 action: right
epoch # 1 / 3000 action: up
epoch # 1 / 3000 action: up
epoch # 1 / 3000 action: up
epoch # 1 / 3000 action: up
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: left
epoch # 1 / 3000 action: up

Won in 28 steps last action: up


[(10, 9, 'left'),
 (9, 9, 'left'),
 (8, 9, 'left'),
 (7, 9, 'up'),
 (7, 8, 'up'),
 (7, 7, 'left'),
 (6, 7, 'left'),
 (5, 7, 'up'),
 (5, 6, 'up'),
 (5, 5, 'left'),
 (4, 5, 'left'),
 (3, 5, 'left'),
 (2, 5, 'left'),
 (1, 5, 'left'),
 (0, 5, 'right'),
 (1, 5, 'right'),
 (2, 5, 'right'),
 (3, 5, 'right'),
 (4, 5, 'right'),
 (5, 5, 'right'),
 (6, 5, 'right'),
 (7, 5, 'up'),
 (7, 4, 'up'),
 (7, 3, 'up'),
 (7, 2, 'up'),
 (7, 1, 'left'),
 (6, 1, 'left'),
 (5, 1, 'up')]

In [25]:
env.render(rewards)


█████O█████
█░░░░░░░░░█
█░█████░█░█
█░░░░░░░█░█
███░███░███
░░░░░░░░░░Ω
█████░█████
█░░░░░░░░░█
███░███░███
░░░░░░░░░░░
███████████
